In [ ]:
import pandas as pd
df = pd.read_csv('site.csv',encoding='gbk')
shanghai_df = df[df['省市自治区'] == '上海市'].copy()
shanghai_df['序号'] = shanghai_df.reset_index().index + 1
shanghai_df.to_csv('shanghai.csv', index=False,encoding='gbk')

In [ ]:
ratings = pd.read_csv('google_review_ratings.csv',encoding='gbk')
display(ratings.head())
cn_column = ["User","宗教场所","度假区","海滩","公园/森林公园","剧院/表演场所","博物馆/展览馆","购物中心/商场","动物园/水族馆","餐厅/美食","酒吧/夜生活","本地服务/体验活动","汉堡/披萨店（需调整或删除）","酒店/住宿","果汁吧/甜品店","艺术画廊/艺术空间","舞蹈俱乐部/娱乐场所","游泳池","健身房/运动中心","面包店/糕点店（需调整或删除）","美容&Spa","咖啡厅（需调整或删除）","观景点","纪念碑/历史遗址","花园/自然景观"]
print(len(cn_column))
ratings.columns= cn_column
ratings.to_csv('ratings_cn.csv',index=False,encoding='gbk')

In [ ]:
# craw pic
import urllib.request
import os
save_dir = 'pic/'
for index, row in shanghai_df.iterrows():
    chinese_name = row['景点名称']
    if os.path.exists(save_dir+f'{row}-{chinese_name}.jpg'):
        continue
    search_url = f'https://www.google.com/search?q={chinese_name}&sca_upv=1&sxsrf=ACQVn08MS_K-XF5g6zaVj8_3i3jeSpNqDA:1708866753932&source=hp&biw=2048&bih=1026&ei=wTzbZbLTNrOb0-kP4Zu6yA8&iflsig=ANes7DEAAAAAZdtK0b9SPr7NNAoGw2MYZ_gBA-HN3NmK&ved=0ahUKEwjyt9beyMaEAxWzzTQHHeGNDvkQ4dUDCA8&oq=%E4%B8%9C%E6%96%B9%E6%98%8E%E7%8F%A0&gs_lp=EgNpbWciDOS4nOaWueaYjuePoDIFEAAYgAQyBRAAGIAEMgUQABiABDIFEAAYgAQyBRAAGIAEMgUQABiABDIFEAAYgAQyBRAAGIAEMgUQABiABDIFEAAYgARI0SdQ3xtY8SJwAXgAkAEAmAGfAaABnwGqAQMwLjG4AQzIAQD4AQL4AQGKAgtnd3Mtd2l6LWltZ5gCAqACpgGoAgrCAgcQIxjqAhgnmAMFkgcDMS4x&sclient=img&udm=2'
    try:
        response = urllib.request.urlopen(search_url)
        page_content = response.read().decode('utf-8')
        start_index = page_content.find('"objURL":"') + 10
        end_index = page_content.find('.jpg"', start_index) + 4
        image_url = page_content[start_index:end_index]
        save_path = save_dir + f'{row}-{chinese_name}.jpg'
        urllib.request.urlretrieve(image_url, save_path)
        print(f'成功保存图片：{save_path}')
    except Exception as e:
        print(f'搜索图片出错：{e}')

In [37]:
from openai import OpenAI
import re
from tqdm import tqdm
import time
client = OpenAI(
    base_url = 'https://api.kwwai.top/v1',
    api_key = 'sk-AYP9eKTcCHOqFrneC4E358Af7f12489dA601A5F012099fD1'
)

def try_get_description(name):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
        {
            "role": "user",
            "content": f"请用markdown格式生成一段对于{name}景点的介绍，要求最后md展示的效果较为精美，重点加粗或者变色。要求直接输出介绍，并且字数越多越好",
        }
    ],
    )
    return response.choices[0].message.content


total_line=len(shanghai_df)
for index, row in tqdm(shanghai_df.iterrows(),total=total_line,desc='Getting description...') :
    chinese_name = row['景点名称']
    if(len(row['景点描述'])>10):
        continue
    description = try_get_description(chinese_name)
    shanghai_df.loc[index, '景点描述'] = description
    shanghai_df.fillna('', inplace=True)
    shanghai_df.to_csv('shanghai.csv', index=False,encoding='gbk', errors='ignore')

Getting description...: 100%|██████████| 432/432 [12:24<00:00,  1.72s/it]


In [56]:
def extract_numbers(string):
    numbers = re.findall(r'\d+', string)
    if len(numbers) == 1:
        return numbers[0]
    else:
        return ','.join(numbers)
def extract_single_numbers(string):
    numbers = re.findall(r'\d+', string)
    return numbers[0]

shanghai_df['分类'] = shanghai_df['分类'].apply(extract_numbers)
shanghai_df['单一分类'] = shanghai_df['分类'].apply(extract_single_numbers)
shanghai_df.to_csv('shanghai.csv', index=False,encoding='gbk', errors='ignore')
shanghai_df

,序号,景点名称,地区,评分,评级,地址,评语,价格,销量,省市自治区,景点描述,分类,单一分类
514,1,东方明珠,上海·上海·浦东新区,0.76,5A景区,上海市浦东新区世纪大道1号,感受云端漫步，品味老上海风情,88.0,5535,上海市,# 东方明珠\n\n东方明珠塔，坐落在上海浦东陆家嘴，是中国最著名的地标之一，也是上海最具代...,22,22
515,2,上海杜莎夫人蜡像馆,上海·上海·黄浦区,0.74,4A景区,上海市黄浦区南京西路2-68号新世界商厦10楼,走自己的红毯，与世界明星相遇,130.0,4022,上海市,### 上海杜莎夫人蜡像馆\n\n上海杜莎夫人蜡像馆位于中国上海市浦东新区世纪大道，是一家展...,6,6
516,3,上海迪士尼乐园,上海·上海·浦东新区,0.00,\N,上海市浦东新区川沙新镇上海迪士尼度假区内,去哪儿付款码迪士尼园区内满120减50,570.0,55113,上海市,### 上海迪士尼乐园景点介绍\n\n上海迪士尼乐园是位于中国上海的迪士尼主题公园，于201...,18,18
517,4,上海野生动物园,上海·上海·浦东新区,0.72,5A景区,上海市浦东新区南六公路178号,与野生动物近距离的“亲密接触”,118.0,7312,上海市,### 上海野生动物园\n\n位于上海市浦东新区的上海野生动物园是一处集观赏、教育和科研于一...,7,7
518,5,上海科技馆,上海·上海·浦东新区,0.74,5A景区,上海市浦东新区世纪大道2000号(近二号线上海科技馆站),技术改变生活，只有想不到没有做不到,51.0,3657,上海市,### 上海科技馆\n\n上海科技馆位于浦东新区，是一座集科普教育、科技展览、科技体验为一体...,6,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2672,428,崇明西沙湿地公园,上海·上海·崇明岛,0.70,4A景区,上海市崇明县崇明岛西南端、明珠湖公园向西 500 米,具有成片滩涂林地的自然湿地,0.0,0,上海市,**崇明西沙湿地公园**是位于上海崇明岛西部的一处独特景点，拥有丰富的自然生态资源和文化遗产...,"13,4,22",13
2673,429,上海多肉植物展,上海·上海·长宁区,0.00,\N,上海市长宁区世博源购物中心（世博大道1368号）L2层空中花园,也是全国无数肉友翘首以盼的嘉年华！2015上海国际多肉展（2015,0.0,0,上海市,上海多肉植物展是一个迷人的园林景点，汇集了各种各样的多肉植物，营造出一种独特的植物世界。展览...,6,6
2674,430,佘山国家森林公园,上海·上海·佘山国家旅游度假区,0.70,4A景区,上海市松江区佘山塔弄口,佘山国家森林公园，位于上海市郊西南松江境内，距市中心30公里，距虹桥机场20公里，北靠318...,0.0,0,上海市,**佘山国家森林公园**\n\n**佘山国家森林公园**，坐落在上海西南部的崇明区，是上海市...,4,4
2675,431,上海新场古镇,上海·上海·浦东新区,0.70,3A景区,上海市浦东新区新场镇海泉街128号,古建筑形态多样，异常华美，独具特色,0.0,0,上海市,**上海新场古镇**\n\n**上海新场古镇**位于上海市浦东新区，是一座历史悠久的古镇，距...,9,9


In [41]:
import re
def try_get_classification(name):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
        {
            "role": "user",
            "content": f'''请将{name}这个景点进行分类，一共有24类，分别如下，直接给出这个景点的分类结果，以数字下标输出：
"宗教场所","度假区","海滩","公园/森林公园","剧院/表演场所","博物馆/展览馆","购物中心/商场","动物园/水族馆","餐厅/美食","酒吧/夜生活","本地服务/体验活动","汉堡/披萨店（需调整或删除）","酒店/住宿","果汁吧/甜品店","艺术画廊/艺术空间","舞蹈俱乐部/娱乐场所","游泳池","健身房/运动中心","面包店/糕点店（需调整或删除）","美容&Spa","咖啡厅（需调整或删除）","观景点","纪念碑/历史遗址","花园/自然景观"

比如：对于 太阳岛国际旅游度假区 这个景点， 应该输出: 2 原因：这个景点属于度假区，而度假区是第2个分类
''',
        }
    ],
    )
    classification = response.choices[0].message.content
    if not re.search(r'\b([1-9]|1[0-9]|2[0-5])\b', classification):
        classification = try_get_classification(name)  # 递归调用直到成功获得符合要求的分类
    return classification
total_line=len(shanghai_df)
for index, row in tqdm(shanghai_df.iterrows(),total=total_line,desc='Getting description...') :
    chinese_name = row['景点名称']
    # if(int(row['分类'])>0):
    #     continue
    classification = try_get_classification(chinese_name)
    shanghai_df.loc[index, '分类'] = classification
    shanghai_df.fillna('', inplace=True)
    shanghai_df.to_csv('shanghai_classify.csv', index=False,encoding='gbk', errors='ignore')

Getting description...: 100%|██████████| 432/432 [09:00<00:00,  1.25s/it]


In [60]:
ratings

,User,宗教场所,度假区,海滩,公园/森林公园,剧院/表演场所,博物馆/展览馆,购物中心/商场,动物园/水族馆,餐厅/美食,...,艺术画廊/艺术空间,舞蹈俱乐部/娱乐场所,游泳池,健身房/运动中心,面包店/糕点店（需调整或删除）,美容&Spa,咖啡厅（需调整或删除）,观景点,纪念碑/历史遗址,花园/自然景观
0,User 1,0.00,0.00,3.63,3.65,5.00,2.92,5.00,2.35,2.33,...,1.74,0.59,0.50,0.00,0.50,0.00,0.00,0.0,0.0,0.00
1,User 2,0.00,0.00,3.63,3.65,5.00,2.92,5.00,2.64,2.33,...,1.74,0.59,0.50,0.00,0.50,0.00,0.00,0.0,0.0,0.00
2,User 3,0.00,0.00,3.63,3.63,5.00,2.92,5.00,2.64,2.33,...,1.74,0.59,0.50,0.00,0.50,0.00,0.00,0.0,0.0,0.00
3,User 4,0.00,0.50,3.63,3.63,5.00,2.92,5.00,2.35,2.33,...,1.74,0.59,0.50,0.00,0.50,0.00,0.00,0.0,0.0,0.00
4,User 5,0.00,0.00,3.63,3.63,5.00,2.92,5.00,2.64,2.33,...,1.74,0.59,0.50,0.00,0.50,0.00,0.00,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5451,User 5452,0.91,5.00,4.00,2.79,2.77,2.57,2.43,1.09,1.77,...,5.00,0.66,0.65,0.66,0.69,5.00,1.05,5.0,5.0,1.56
5452,User 5453,0.93,5.00,4.02,2.79,2.78,2.57,1.77,1.07,1.76,...,0.89,0.65,0.64,0.65,1.59,1.62,1.06,5.0,5.0,1.09
5453,User 5454,0.94,5.00,4.03,2.80,2.78,2.57,1.75,1.05,1.75,...,0.87,0.65,0.63,0.64,0.74,5.00,1.07,5.0,5.0,1.11
5454,User 5455,0.95,4.05,4.05,2.81,2.79,2.44,1.76,1.03,1.74,...,5.00,0.64,0.63,0.64,0.75,5.00,1.08,5.0,5.0,1.12


In [65]:
import numpy as np
final_ratings_data = []

for idx, user_row in tqdm(ratings.iterrows(),total=len(ratings),desc="rating...") :
    user_id = user_row['User']
    # if idx >10:
    #     break
    for category_index, category_name in enumerate(ratings.columns[1:], start=1):  # Skip 'User' column, index starts at 1
        try:
            target_mean_score = float(user_row[category_name])
        except (ValueError, TypeError):
            target_mean_score = float(0)
        spots_in_category = shanghai_df[shanghai_df['单一分类'] == str(category_index)]['序号'].tolist()
        # Determine the number of spots to rate (20%-50%)
        num_spots_to_rate = np.random.randint(low=int(len(spots_in_category) * 0.2), high=int(len(spots_in_category) * 0.5) + 1)
        selected_spots = np.random.choice(spots_in_category, size=num_spots_to_rate, replace=False)

        # Generate ratings with consideration to the target average score
        # Simplified approach: Adjust the range of possible ratings based on the target average score
        if target_mean_score <= 2:
            spot_ratings = [np.random.randint(1, 4) for _ in range(num_spots_to_rate)]
        elif target_mean_score <= 3:
            spot_ratings = [np.random.randint(2, 5) for _ in range(num_spots_to_rate)]
        else:
            spot_ratings = [np.random.randint(3, 6) for _ in range(num_spots_to_rate)]

        # Add the ratings to the final data list
        for spot_id, spot_rating in zip(selected_spots, spot_ratings):
            final_ratings_data.append([user_id, spot_id, spot_rating])


final_ratings_df = pd.DataFrame(final_ratings_data, columns=['用户ID', '景点ID', '评分'])
final_ratings_df.to_csv('user_rating.csv', index=False,encoding='gbk', errors='ignore')
final_ratings_df.head()

rating...: 100%|██████████| 5456/5456 [00:42<00:00, 126.90it/s]


,用户ID,景点ID,评分
0,User 1,403,2
1,User 1,350,1
2,User 1,152,1
3,User 1,221,1
4,User 1,421,1
